#### LlamaIndex Document Class

### Notes

In [3]:
from llama_index.core import Document
mydocument = Document(text="Hello LlamaIndex")
mydocument.dict()

{'id_': '9f9e2787-a6e2-48f2-a117-a0c5656e9bff',
 'embedding': None,
 'metadata': {},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {},
 'metadata_template': '{key}: {value}',
 'metadata_separator': '\n',
 'text_resource': {'embeddings': None,
  'text': 'Hello LlamaIndex',
  'path': None,
  'url': None,
  'mimetype': None},
 'image_resource': None,
 'audio_resource': None,
 'video_resource': None,
 'text_template': '{metadata_str}\n\n{content}',
 'class_name': 'Document',
 'text': 'Hello LlamaIndex'}

#### Using Simple Directory Loader

• Import **SimpleDirectoryReader**

    from llama_index.core import SimpleDirectoryReader

• **Load all the files in my _folder**

    documents = SimpleDirectoryReader("my_folder").load_data()

• **Load all the files in my _folder and all its subdirectories**

    documents = SimpleDirectoryReader("my_folder", recursive=True).load_data()

• **Load specific files using the input_files parameter**

    documents = SimpleDirectoryReader(input_files=["my_folder/some_text.txt"]).load_data()

• **Load specific file types using the required_ext parameter**

    documents = SimpleDirectoryReader(input_dir="my_folder" ,
                required_exts=[".txt",".Csv"]).load _data()

#### Using LlamaIndex nodes

• LlamaIndex nodes are simply text chunks

• LlamaIndex provides an easy-to-use text chunker called **SentenceSplitter**.

  *Import sentence SentenceSplitter from llama_index.core.node:*

    from llama_index.core.node_parser import SentenceSplitter

• **SentenceSplitter** splits long texts recursively based on specific characters, such as newline characters and periods.

• Define the node parser as an instance of SentenceSplitter

• Pass the chunk_size parameter to control the max length of chunks

• Pass in the chunk_overlap parameter to control the max overlap between chunks

    node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=20)


To use SentenceSplitter:

• Use the get_nodes_from_documents method to split documents into individual nodes

    nodes = node_parser.get_nodes_from_documents(documents,show_progress=False)

• The node parser returns a list of LlamaIndex TextNode instances, which are similar in structure to Document instances

#### Generate and store embeddings persistently

For simple use cases where vectors are generated using a default model
and stored in-memory:

        from llama_index.core import VectorStoreIndex
        index = VectorStoreIndex (nodes)

For complex cases with custom models and persistent storage:

1. Import libraries:

        import chromadb
        from llama_index.core import VectorStoreIndex, StorageContext
        from llama_index.vector_stores.chroma import ChromaVectorStore
        from llama_index.embeddings.huggingface import HuggingFaceEmbedding

2. Define the embedding model:

        embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

3. Set up the vector store and storage context:

        db = chromadb.PersistentClient(path="chroma_db_test")
        chroma_collection = db.get_or_create_collection ("mycollection")
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)

4. Pass the nodes, the model, and storage context to VectorStoreIndex:

        index = VectorStoreIndex(nodes=nodes,embed _model=embed_model,
        storage_context=storage_context)

#### Prompt embedding and retrieval steps

* Create the retriever by calling the as_retriever method on the VectorStoreIndex object.

        retriever = index.as_retriever()
        retrieved_nodes = retriever.retrieve("User's prompt")


#### LlamaIndex's response synthesizer

* LlamaIndex uses a response synthesizer to combine prompt augmentation,
LLM querying, and response regeneration.

* Given a user's prompt and retrieved nodes, the response synthesizer's
synthesize method generates a response from the LLM.

      from llama_index.core import get_ response_synthesizer
      response_synthesizer = get_response_synthesizer()
      response = response_synthesizer.synthesize(query="User's prompt,nodes=retrieved_nodes)

#### LlamaIndex's query engine



• LlamaIndex's query engine combines the prompt embedding, retrieval, prompt
augmentation, LLM querying, and response generation steps.

• Given a user's prompt, the query engine's query method generates a response
from the LLM.

      query_engine = index.as_query_engine()
      response = query_engine.query ("User's prompt")

## Installs and Setup

In [1]:
from dotenv import load_dotenv
#from llama_index.llms.gemini import Gemini
#import google.generativeai as genai
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from IPython.display import Markdown, display
from llama_index.core import ServiceContext
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
import chromadb
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.llms.groq import Groq

/Users/praveennair/Learning/Gen AI/AI_Projects/RAG_LlamaIndex/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
load_dotenv() 
gemini_api_key = os.getenv("GEMINI_API_KEY")

if not gemini_api_key:
	raise ValueError("GEMINI_API_KEY not found in environment variables. Please set it in your .env file.")

llm = GoogleGenAI(model="gemini-2.0-flash-lite", temperature=0.2, api_key=gemini_api_key)


In [2]:
import os
load_dotenv() 
groq_api_key = os.getenv("GROQ_API_KEY")
if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found in environment variables. Please set it in your .env file.")  
 
llm = Groq(model="llama-3.3-70b-versatile", temperature=0.2, api_key=groq_api_key)


In [3]:
persist_directory = "./chroma_db"

# Check if the directory exists
if not os.path.exists(persist_directory):
    # If it doesn't exist, create it
    print(f"Creating directory: {persist_directory}")
    os.makedirs(persist_directory)

    documents=SimpleDirectoryReader(input_files=["Power+BI+Ebook.pdf"])
    doc=documents.load_data()   

    node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=100)
    nodes = node_parser.get_nodes_from_documents(doc,show_progress=False)

    embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

    db = chromadb.PersistentClient(path=persist_directory)
    chroma_collection = db.get_or_create_collection ("mycollection")
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    index = VectorStoreIndex(nodes=nodes,embed_model=embed_model,storage_context=storage_context)
    
else:
    print(f"Directory already exists: {persist_directory}")
    embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = chromadb.PersistentClient(path=persist_directory)
    chroma_collection = db.get_or_create_collection("mycollection")
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

  # Load the VectorStoreIndex from the existing vector store
    index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store,
        embed_model=embed_model # Ensure embed_model is passed
    )
    print("Index loaded from ChromaDB.")

Directory already exists: ./chroma_db
Index loaded from ChromaDB.


* Use Query Engine to query

In [4]:
query_engine = index.as_query_engine(llm=llm)
query_engine.query ("What are PowerBI Desktop Features?").response

/Users/praveennair/Learning/Gen AI/AI_Projects/RAG_LlamaIndex/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


'The features of Power BI Desktop include:\n\n1. Get Data: Easily connect, clean, and mashup data from 80+ data sources, with live connectivity, custom data connectors, and data preparation using Power Query.\n2. Analyze: Build powerful models and flexible measures with automatic model creation, high-performance engine, point-and-click analysis, and DAX formulas.\n3. Visualise: Create stunning interactive reports with 150+ visuals, drag-drop canvas, multiple interactive visualizations, custom visuals, and data storytelling with bookmarks and navigation.\n4. Publish: Share insights with others by publishing directly to the cloud or on-premises, with automatic data refresh, report packaging in apps, and analytics content management.\n5. Collaborate: Empower organization with self-service analytics, allowing users to collaborate and share reports and dashboards with other team members.'

* Using a custom prompt template

In [5]:
from llama_index.core import PromptTemplate,ChatPromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole

prompt_str= """
            "You are a helpful assistant that answers questions based on the provided context. "
            "If the context doesn't contain the answer, state that you don't know. "
            "Be concise and do not add extraneous information."
            Context information is below.
            ---------------------
            {context_str}
            ---------------------
            {query_str}
            """
message_template = ChatMessage(content=prompt_str, role=MessageRole.USER)
query_prompt=ChatPromptTemplate([message_template])

In [6]:
query_engine = index.as_query_engine(llm=llm,query_template=query_prompt)
query_engine.query ("What are building blocks of PowerBI?").response

'The building blocks of Power BI include visualizations, datasets, reports, dashboards, and tiles. Visualizations represent data in a visual format, such as charts or graphs. Datasets are collections of data used to create visualizations. Reports are collections of visualizations, while dashboards are single-page interfaces that use key elements from reports to tell a story. Tiles are individual visualizations found in reports or on dashboards.'

In [7]:
query_engine.query("What is RLS?").response

"Row-Level Security (RLS) is a feature that enables you to restrict data access based on a user's role or permission level, allowing you to control what data users can see, interact with, and modify within a report or dashboard."